In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import itertools
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import *
import pandas as pd
from sklearn.preprocessing import normalize

"""Change to the data folder"""
new_path = "./new_train/new_train"
#new_path = r"C:\Users\mnisyu\Desktop\CSE151B competition\cse151b-spring\\"
#new_path = ".."

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/cse151b-spring.zip

Streaming output truncated to the last 5000 lines.
  inflating: new_train/new_train/9838.pkl  
  inflating: new_train/new_train/98380.pkl  
  inflating: new_train/new_train/98381.pkl  
  inflating: new_train/new_train/98382.pkl  
  inflating: new_train/new_train/98383.pkl  
  inflating: new_train/new_train/98384.pkl  
  inflating: new_train/new_train/98385.pkl  
  inflating: new_train/new_train/98386.pkl  
  inflating: new_train/new_train/98387.pkl  
  inflating: new_train/new_train/98388.pkl  
  inflating: new_train/new_train/98389.pkl  
  inflating: new_train/new_train/9839.pkl  
  inflating: new_train/new_train/98390.pkl  
  inflating: new_train/new_train/98391.pkl  
  inflating: new_train/new_train/98392.pkl  
  inflating: new_train/new_train/98393.pkl  
  inflating: new_train/new_train/98394.pkl  
  inflating: new_train/new_train/98395.pkl  
  inflating: new_train/new_train/98396.pkl  
  inflating: new_train/new_train/98397.pkl  
  inflating: new_train/new_train/98398.pkl  
  infl

In [ ]:
import pickle

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_data  = ArgoverseDataset(data_path="/content/new_train/new_train/")
test_data  = ArgoverseDataset(data_path="/content/new_val_in/new_val_in/")

In [ ]:
batch_sz = 300
def my_collate(batch):
    indexs = [np.where(scene['agent_id'] == scene['track_id'][:,0].reshape(60))[0][0] for scene in batch]
    p_in= [scene['p_in'][index] for (scene,index) in zip(batch, indexs)]
    v_in = [scene['v_in'][index] for (scene,index) in zip(batch, indexs)]
    inp = [np.dstack((p[:,0],p[:,1],v[:,0],v[:,1])).reshape(-1) for(p,v) in zip(p_in,v_in)]
    
    p_out = [scene['p_out'][index] for (scene,index) in zip(batch, indexs)]
    out = [np.dstack((p[:,0],p[:,1])).reshape(-1) for p in p_out]
    
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    
    return [inp, out]

train_iter = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:
train_in, train_label = next(iter(train_iter))
train_in.size()

torch.Size([300, 76])

In [ ]:
train_in

tensor([[3235.3953, 1929.4937, 3235.8562,  ..., 1936.6877, 3243.8857,
         1937.1260],
        [ 818.1125, 2111.6633,  818.2941,  ..., 2111.0044,  820.8439,
         2110.8589],
        [3042.8130, 1613.6917, 3041.8860,  ..., 1602.8489, 3029.0635,
         1602.3530],
        ...,
        [2663.1941, 1406.2654, 2662.9858,  ..., 1403.4619, 2660.0491,
         1403.3490],
        [ 731.8400, 2433.8223,  731.8821,  ..., 2416.4285,  732.2079,
         2415.3169],
        [3138.4736, 1705.7502, 3137.8962,  ..., 1698.3250, 3128.7822,
         1697.6072]])

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hid1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hid2 = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hid1(x))
        x = self.hid2(x)            # linear output length 60
        return x

In [ ]:
net = Net(n_feature=76, n_hidden=1000, n_output=60).to(device)
# print(net)  # network architecture
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=0.01#)

In [ ]:
len(train_iter)

687

In [ ]:
import time
start_time = time.time()
for epoch in range(20):  # loop over the dataset multiple time
    running_loss = 0.0
    for i, data in enumerate(train_iter, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            #print(outputs[0])
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
        
print("--- %s seconds ---" % (time.time() - start_time))

[1,    50] loss: 612135.679
[1,   100] loss: 3725.073
[1,   150] loss: 220.306
[1,   200] loss: 70.264
[1,   250] loss: 45.078
[1,   300] loss: 36.430
[1,   350] loss: 30.877
[1,   400] loss: 26.944
[1,   450] loss: 23.834
[1,   500] loss: 42.487
[1,   550] loss: 24.586
[1,   600] loss: 17.354


In [ ]:
import time
start_time = time.time()
for epoch in range(20):  # loop over the dataset multiple time
    running_loss = 0.0
    for i, data in enumerate(train_iter, 0):
        if i < 480: 
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            #print(outputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 50 == 49:    # print every 50 mini-batches
                #print(outputs[0])
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 50))
                running_loss = 0.0
        else: 
            with torch.no_grad():
              val_loss = 0
              for j, batch in enumerate(train_iter, 0):
                  inputs, labels = batch
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  # zero the parameter gradients
                  #optimizer.zero_grad()

                  # forward + backward + optimize
                  outputs = net(inputs)
                  #print(outputs)
                  loss = loss_func(outputs, labels)
                  #loss.backward()
                  #optimizer.step()

                  # print statistics
                  val_loss += loss.item()
              print('val loss ', epoch, ' ', val_loss / 207)
              break
    #scheduler.step()
print("--- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: ignored

# Real Model starts from here, aboves are explorations

In [ ]:
import pickle

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_data  = ArgoverseDataset(data_path="/content/new_train/new_train/")
test_data  = ArgoverseDataset(data_path="/content/new_val_in/new_val_in/")

array([3235.39526367, 1929.49365234, 3235.85620117, 1929.80456543,
       3235.94506836, 1929.90905762, 3237.28198242, 1931.51367188,
       3237.79418945, 1931.74328613, 3238.26123047, 1932.32592773,
       3238.71533203, 1932.63842773, 3239.05664062, 1932.74816895,
       3239.63549805, 1933.54748535, 3240.15014648, 1934.04931641,
       3240.55395508, 1934.35949707, 3241.04003906, 1934.9621582 ,
       3241.54272461, 1935.44311523, 3242.02246094, 1935.65124512,
       3242.36499023, 1935.9407959 , 3242.83886719, 1936.31408691,
       3243.12109375, 1936.5065918 , 3243.38110352, 1936.68774414,
       3243.88574219, 1937.12597656])

In [ ]:
len(train_data)

205942

# Try Seq2Seq

In [ ]:
batch_sz = 300
def my_collate(batch):
    indexs = [np.where(scene['track_id'][:,0,:] == scene['agent_id'])[0][0] for scene in batch]
    p_in= [scene['p_in'][index].flatten() for (scene,index) in zip(batch, indexs)]
    v_in = [scene['v_in'][index].flatten() for (scene,index) in zip(batch, indexs)]
    #inp = [np.dstack((normalize(p[:,0].reshape(-1, 1)),normalize(p[:,1].reshape(-1, 1)),normalize(v[:,0].reshape(-1, 1)),normalize(v[:,1].reshape(-1, 1)))).reshape(-1) for(p,v) in zip(p_in,v_in)]
    inp = np.concatenate((p_in, v_in), axis = 1)

    out = [scene['p_out'][index].flatten() for (scene,index) in zip(batch, indexs)]
    
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    
    
    return [inp, out]

train_iter = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)


In [ ]:
def test_collate(batch):
    indexs = [np.where(scene['track_id'][:,0,:] == scene['agent_id'])[0][0] for scene in batch]
    p_in= [scene['p_in'][index].flatten() for (scene,index) in zip(batch, indexs)]
    v_in = [scene['v_in'][index].flatten() for (scene,index) in zip(batch, indexs)]
    #inp = [np.dstack((normalize(p[:,0].reshape(-1, 1)),normalize(p[:,1].reshape(-1, 1)),normalize(v[:,0].reshape(-1, 1)),normalize(v[:,1].reshape(-1, 1)))).reshape(-1) for(p,v) in zip(p_in,v_in)]
    inp = np.concatenate((p_in, v_in), axis = 1)

    inp = torch.FloatTensor(inp)
    return inp

test_iter = DataLoader(test_data,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)

In [ ]:
test_in = next(iter(test_iter))
test_in.size()

torch.Size([300, 76])

In [ ]:
train_in, train_label = next(iter(train_iter))
train_label.size()

torch.Size([300, 60])

In [ ]:
train_in.size()

torch.Size([300, 76])

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hid1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hid2 = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hid1(x))
        x = self.hid2(x)            # linear output length 60
        return x


In [ ]:
device = torch.device('cuda')
#net = Net(n_feature=76, n_hidden=500, n_output=60).to(device)
net = nn.Sequential(nn.Linear(76, 60))
net.to(device)

# print(net)  # network architecture
optimizer = torch.optim.Adam(net.parameters(), lr=0.00003)
loss_func = torch.nn.MSELoss()

In [ ]:
import time
start_time = time.time()
for epoch in range(10):  # loop over the dataset multiple time
    running_loss = 0.0
    for i, data in enumerate(train_iter, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 50 mini-batches
            #print(outputs[0])
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

[1,   200] loss: 13.116
[1,   400] loss: 12.974
[1,   600] loss: 13.049
[2,   200] loss: 12.705
[2,   400] loss: 12.563
[2,   600] loss: 12.638
[3,   200] loss: 12.307
[3,   400] loss: 12.178
[3,   600] loss: 12.251
[4,   200] loss: 11.932


KeyboardInterrupt: ignored

In [ ]:
import time
start_time = time.time()
for epoch in range(4):  # loop over the dataset multiple time
    running_loss = 0.0
    for i, data in enumerate(train_iter, 0):
        if i < 480: 
            pass
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            #print(outputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 50 == 49:    # print every 50 mini-batches
                #print(outputs[0])
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 50))
                running_loss = 0.0
        else: 
            with torch.no_grad():
              val_loss = 0
              for j, batch in enumerate(train_iter, 0):
                  inputs, labels = batch
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  # zero the parameter gradients
                  #optimizer.zero_grad()

                  # forward + backward + optimize
                  outputs = net(inputs)
                  #print(outputs)
                  loss = loss_func(outputs, labels)
                  #loss.backward()
                  #optimizer.step()

                  # print statistics
                  val_loss += loss.item()
              print('val loss ', epoch, ' ', val_loss / 207)
              break 
print("--- %s seconds ---" % (time.time() - start_time))

[1,    50] loss: 11.088
[1,   100] loss: 11.087
[1,   150] loss: 10.799
[1,   200] loss: 10.859
[1,   250] loss: 11.178
[1,   300] loss: 10.605
[1,   350] loss: 10.859
[1,   400] loss: 10.939
[1,   450] loss: 11.029
val loss  0   36.11944948878265
[2,    50] loss: 10.907
[2,   100] loss: 10.900
[2,   150] loss: 10.625
[2,   200] loss: 10.685
[2,   250] loss: 11.001
[2,   300] loss: 10.440
[2,   350] loss: 10.690
[2,   400] loss: 10.771
[2,   450] loss: 10.860
val loss  1   35.55599980423416
[3,    50] loss: 10.740
[3,   100] loss: 10.727
[3,   150] loss: 10.464
[3,   200] loss: 10.524
[3,   250] loss: 10.837
[3,   300] loss: 10.288
[3,   350] loss: 10.533
[3,   400] loss: 10.615
[3,   450] loss: 10.702
val loss  2   35.03872620080404
[4,    50] loss: 10.585
[4,   100] loss: 10.567
[4,   150] loss: 10.315
[4,   200] loss: 10.375
[4,   250] loss: 10.685
[4,   300] loss: 10.147
[4,   350] loss: 10.388
[4,   400] loss: 10.470
[4,   450] loss: 10.557
val loss  3   34.56117248535156
--- 2348

In [ ]:
MODEL_PATH = '/content/model1.pth' # save model
torch.save(net, MODEL_PATH)

In [ ]:
net = torch.load(MODEL_PATH)
net

Net(
  (hid1): Linear(in_features=76, out_features=2000, bias=True)
  (hid2): Linear(in_features=2000, out_features=60, bias=True)
)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
subission = submission[0:0]

In [ ]:
predictions = []
for i, data in enumerate(test_iter, 0):
    data = data.to(device)
    pred = net(data)
    predictions.append(pred)

In [ ]:
predictions_lst = []
for batch in predictions:
    for scene in batch:
        predictions_lst.append(scene.tolist())
scene_id = submission['ID']
scene_id = scene_id.tolist()
for i in range(len(predictions_lst)):
    predictions_lst[i].insert(0, scene_id[i])

first_submission = pd.DataFrame(predictions_lst, columns = submission.columns)
first_submission.to_csv('/content/fc_submission11.csv', index=False)
print('done!')

done!


In [ ]:
first_submission = pd.DataFrame(predictions_lst, columns = submission.columns)
first_submission.to_csv('/content/fc_submission5.csv', index=False)

In [ ]:
len(predictions_lst)

3200

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]
        #print('after shape', out.size())

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        #print('output shape', out.size())
        return out
    
def get_model(model, model_params):
    models = {
        "gru": GRUModel
    }
    return models.get(model.lower())(**model_params)


class Optimization:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
    
    def train_step(self, x, y):
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        #print(y.size())
        #print(yhat.unsqueeze(2).size())
        yhat = yhat.unsqueeze(2)
        
        #print(y)
       # print(yhat)
        
        loss = self.loss_fn(y, yhat)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()
    
    def train(self, train_loader, val_loader, batch_size=64, n_epochs=50, n_features=1):

        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            count = 0
            losses = 0
            for x_batch, y_batch in train_loader:
                count += 1
                x_batch = x_batch.view([batch_size, -1, n_features]).to(device)
                y_batch = y_batch.to(device)
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
                losses += loss
                
                if count % 100 == 0: 
                    print(epoch, ' ', count, ' ', losses / count)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            

            if (epoch <= 10) | (epoch % 50 == 0):
                print(
                    training_loss
                )

        torch.save(self.model.state_dict(), model_path)


In [ ]:
def evaluate(self, test_loader, batch_size=1, n_features=1):
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features]).to(device)
                y_test = y_test.to(device)
                self.model.eval()
                yhat = self.model(x_test)
                predictions.append(yhat.to(device).detach().numpy())
                values.append(y_test.to(device).detach().numpy())

        return predictions, values

In [ ]:
input_dim = 1
output_dim = 60
hidden_dim = 64
layer_dim = 3
batch_size = 64
dropout = 0.2
n_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-6
device = torch.device('cuda')

model_params = {'input_dim': input_dim,
                'hidden_dim' : hidden_dim,
                'layer_dim' : layer_dim,
                'output_dim' : output_dim,
                'dropout_prob' : dropout}

model = get_model('gru', model_params)
model = model.to(device)

loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
opt.train(train_iter, train_iter,batch_size=batch_sz, n_epochs=n_epochs, n_features=input_dim)

1   100   4646535.105
1   200   4643376.17
1   300   4636715.743333333
1   400   4623901.015


In [ ]:
predictions, values = opt.evaluate(test_loader_one, batch_size=1, n_features=input_dim)

In [ ]:
class Encoder(nn.Module): 
    def __init__(self, input_size, hidden_size, num_layers, p): 
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout = p, batch_first = True)
        
    def forward(self, x): 
        # x shape: (seq,batch)
        outputs, (hidden, cell) = self.rnn(x)
        #print('output', outputs.size())
        #print('hidden', hidden.size())
        
        return hidden, cell

In [ ]:
class Decoder(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size, num_layers, p):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout=p, batch_first = True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden, cell): 
        # shape of x: (N), we want (1, N)
        #print('before', x.size())
        x = x.unsqueeze(0)
        
        #print('decoder here')
        #print(x.size())
        #print('hidden', hidden.size())
        #print('cell', cell.size())
        outputs, (hidden, cell) = self.rnn(x, (hidden, cell))
        # (1, N, hidden_size)
        prediction = self.fc(outputs)
        # (1, N, output_size)
        
        predictions = prediction.squeeze(0)
        
        return predictions, hidden, cell
        

In [ ]:
class Seq2Seq(nn.Module): 
    def __init__(self, encoder, decoder): 
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, source, target, teacher_force_ratio=0.5): 
        batch_size = source.shape[0]
        target_len = target.shape[1]
        target_output = 30
        
        outputs = []
        #print(source.size(-1))
        #print(source.size())
        hidden, cell = self.encoder(source)
        
        x = target[:, 0, :]
        #print('target', x.size())
        (10, 60, 19, 4)
        (10, 60, 19*4)
        (10, 60, 30*4)
        (10, 19, 4)
        
        for t in range(1, target_len): 
            print('start')
            print('decoder input', x.size())
            print('hidden', hidden.size())
            print('cell', cell.size())
            output, hidden, cell = self.decoder(x, hidden, cell)
            print('output', output)
            outputs.append(output)
            
            x = target[t] if np.random.uniform() < teacher_force_ratio else output
            
        return outputs

In [ ]:
num_epochs = 20
leanring_rate = 0.001

load_model = False
input_size_encoder = 38
input_size_decoder = 30
output_size = 2
hidden_size = 1024
num_layers = 2
enc_dropout = 0.1
dec_dropout = 0.1

In [ ]:
encoder_net = Encoder(input_size_encoder, hidden_size, num_layers, enc_dropout)
decoder_net = Decoder(input_size_decoder, hidden_size, output_size, num_layers, dec_dropout)
model = Seq2Seq(encoder_net, decoder_net)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs, labels)
        loss = criterion(outputs, labels)
        a = labels
        b = outputs
        loss.backward()
        optimizer.step()

        # print statistics
        print(loss.item())
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0


38
torch.Size([300, 2, 38])
output torch.Size([300, 2, 1024])
hidden torch.Size([2, 300, 1024])
target torch.Size([300, 30])
start
decoder input torch.Size([300, 30])
hidden torch.Size([2, 300, 1024])
cell torch.Size([2, 300, 1024])
before torch.Size([300, 30])
decoder here
torch.Size([1, 300, 30])
hidden torch.Size([2, 300, 1024])
cell torch.Size([2, 300, 1024])


RuntimeError: Expected hidden[0] size (2, 1, 1024), got [2, 300, 1024]

In [ ]:
a = train_in.reshape(-1, batch_sz, 1)
b = train_label.reshape(-1, batch_sz, 1)

In [ ]:
def my_collate(batch): 
    inp = [np.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [np.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]
train_loader = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:
train_in.size()
train_label.size()

In [ ]:
model(train_in, train_label)

RuntimeError: input.size(-1) must be equal to input_size. Expected 2, got 38

In [ ]:
b = train_label.reshape(-1, batch_sz, 1)
b.size()

In [ ]:
np.random.uniform()

In [ ]:
b = train_label.reshape(-1, batch_sz, 1)
decoder_net(b, hidden, cell)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs, labels)
        loss = criterion(outputs, labels)
        a = labels
        b = outputs
        loss.backward()
        optimizer.step()

        # print statistics
        print(loss.item())
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0


In [ ]:
def my_collate(batch):
    inp = []
    for scene in batch:
        a = []
        agent_id = scene['agent_id']
        agent_idx = np.where(scene['track_id'] == agent_id)[0][0]
        p_input = scene['p_in'][agent_idx]
        inp.append([item for sublist in p_input for item in sublist])
    out = []
    for scene in batch:
        agent_id = scene['agent_id']
        agent_idx = np.where(scene['track_id'] == agent_id)[0][0]
        p_out = scene['p_out'][agent_idx]
        out.append([item for sublist in p_out for item in sublist])
    
    inp = torch.FloatTensor(inp)
    # size: batch size x 38 (linear p_in)
    
    out = torch.FloatTensor(out)
    # size: batch size x 60 (linear p_out)
    
    return [inp, out]

batch_sz = 32
train_loader = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:
class Encoder(nn.Module): 
    def __initz__(self, input_size, embedding_size, hidden_size, num_layers, p): 
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
        
    def forward(self,x): 
        # x shape: (seq_length, N)
        embedding = self.dropout(self.embedding(x))
        outputs, (hidden, cell) = self.rnn(embedding)
        
        return hidden, cell

In [ ]:
class Decoder(nn.Module): 
    def __init__:(self, input_size, hidden_size, output_size, num_layers): 
            super(Decoder, self).__init__()
            self.hidden_size = hidden_size
            self.num_layers = num_layers
            
            ## hidden size should be the same as Encoder
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout=p)
            self.fc = nn.Linear(hidden_size, output_size)
            
        def forward(self, x, hidden, cell): 
            outputs, (hidden , cell) = self.rnn(input_size, (hidden,cell))
            
            predictions = self.fc(outputs)

In [ ]:
import gc
import numpy as np
p_in_x = []
p_in_y = []
p_out_x = []
p_out_y = []
v_in_x = []
v_in_y = []
lane_x = []
lane_y = []
lane_norm_x = []
lane_norm_y = []
count = 0
agent_x = []
agent_y = []
for scene in train_data: 
    count += 1
    p_in_x.append(scene['p_in'][:,:,0].reshape(60*19))
    p_in_y.append(scene['p_in'][:,:,1].reshape(60*19))
    p_out_x.append(scene['p_out'][:,:,0].reshape(60*30))
    p_out_y.append(scene['p_out'][:,:,1].reshape(60*30))
    v_in_x.append(scene['v_in'][:,:,0].reshape(60*19))
    v_in_y.append(scene['v_in'][:,:,1].reshape(60*19))
    
    index = np.where(scene['agent_id'] == scene['track_id'][:,0].reshape(60))[0][0]
    agent_x.append(scene['v_in'][index][:,0])
    agent_y.append(scene['v_in'][index][:,1])
    
    gc.collect()
    
    #lane_x.append(scene['lane'][:,0])
    #lane_y.append(scene['lane'][:,1])
    #lane_norm_x.append(scene['lane_norm'][:,0])
    #lane_norm_y.append(scene['lane_norm'][:,1])
    
    if count % 2000 == 1999: 
        print(count)

In [ ]:
a = np.reshape(p_in_x,(1, 60*19*205912))

In [ ]:
p_in_x

In [ ]:
plt.hist(a['lane_norm'][:,0])
plt.title('Lane Norm x axis for all agents')
plt.xlabel('x axis')
plt.ylabel('Count')

In [ ]:
plt.hist(a['lane_norm'][:,1])
plt.title('Lane Norm Y axis for all agents')
plt.xlabel('y axis')
plt.ylabel('Count')

In [ ]:
import numpy as np
np.where(a['agent_id'] == a['track_id'])

In [ ]:
b = a['v_in'][5]
plt.plot(b[:,0], label='X axis')
plt.plot(b[:,1], label="Y axis")
plt.legend()
plt.title('Velocity Distribution of Target Agent')
plt.xlabel("Time Steps")
plt.ylabel("Velocity")

In [ ]:
plt.plot(a['v_in'][0][:,0], label="agent 1")
plt.plot(a['v_in'][1][:,0], label="agent 2")
plt.plot(a['v_in'][2][:,0], label="agent 3")
plt.plot(a['v_in'][3][:,0], label="agent 4")
plt.plot(a['v_in'][4][:,0], label="agent 5")
plt.legend(loc='upper right')
plt.title('Velocity x axis for all input agents')
plt.xlabel('Time stemps')
plt.ylabel('Velocity')
plt.show()

In [ ]:
plt.plot(a['v_in'][0][:,1], label="agent 1")
plt.plot(a['v_in'][1][:,1], label="agent 2")
plt.plot(a['v_in'][2][:,1], label="agent 3")
plt.plot(a['v_in'][3][:,1], label="agent 4")
plt.plot(a['v_in'][4][:,1], label="agent 5")
plt.legend(loc='upper right')
plt.title('Velocity y axis for all input agents')
plt.xlabel('Time stemps')
plt.ylabel('Velocity')
plt.show()

### Create a loader to enable batch processing

# By Lehan Li

# Simple CNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import itertools
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

batch_sz = 100

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [np.dstack([scene['p_in'][:,:,0].reshape(60*19),scene['p_in'][:,:,1].reshape(60*19),scene['v_in'][:,:,0].reshape(60*19),scene['v_in'][:,:,1].reshape(60*19)]).reshape(60*19,4) for scene in batch]
    
    indexs = [np.where(scene['agent_id'] == scene['track_id'][:,0].reshape(60))[0][0] for scene in batch]
    #p_in= [scene['p_in'][index] for (scene,index) in zip(batch, indexs)]
    #v_in = [scene['v_in'][index] for (scene,index) in zip(batch, indexs)]
    #inp = [np.concatenate((p[:,0],p[:,1],v[:,0],v[:,1])).reshape(76,1) for(p,v) in zip(p_in,v_in)]
    
    p_out = [scene['p_out'][index] for (scene,index) in zip(batch, indexs)]
    v_out = [scene['v_out'][index] for (scene,index) in zip(batch, indexs)]
    out = [np.concatenate((p[:,0],p[:,1])) for p in p_out]
    
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

train_loader = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0, drop_last=True)

In [ ]:
_, (example_datas, labels) = next(enumerate(train_loader))
example_datas.size()

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv1d(4, 8, 1)
        self.conv2 = torch.nn.Conv1d(8, 16, 1)
        self.conv3 = torch.nn.Conv1d(16, 32, 1)
        
        self.linear1 = torch.nn.Linear(32*1140, 8*1140)
        self.linear3 = torch.nn.Linear(8*1140, 1140)
        self.linear5 = torch.nn.Linear(1140, 60)

        self.drop = torch.nn.Dropout(0.2)
    def forward(self, inputs):
        x = inputs
        x = F.relu(self.conv1(x))
        x = self.drop(x)
        x = F.relu(self.conv2(x))
        x = self.drop(x)
        x = self.conv3(x)
        x = x.reshape(100, 32*1140)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear3(x))
        x = self.drop(x)
        x = F.relu(self.linear5(x))
        return x.reshape(-1,60)
    
net = Net()

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.005)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    running_loss_stemp = 0.0
    nstemp = 0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.reshape(batch_sz, 4, -1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs)
        #print(labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(loss.item())
        # print statistics
        running_loss += loss.item()
        running_loss_stemp += loss.item()
        nstemp = i
        
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

# LSTM

In [ ]:
import itertools
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

batch_sz = 1

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [np.dstack([scene['p_in'][:,:,0].reshape(60*19),scene['p_in'][:,:,1].reshape(60*19),scene['v_in'][:,:,0].reshape(60*19),scene['v_in'][:,:,1].reshape(60*19)]).reshape(60*19,4) for scene in batch]
    
    indexs = [np.where(scene['agent_id'] == scene['track_id'][:,0].reshape(60))[0][0] for scene in batch]
    #p_in= [scene['p_in'][index] for (scene,index) in zip(batch, indexs)]
    #v_in = [scene['v_in'][index] for (scene,index) in zip(batch, indexs)]
    #inp = [np.concatenate((p[:,0],p[:,1],v[:,0],v[:,1])).reshape(76,1) for(p,v) in zip(p_in,v_in)]
    
    p_out = [scene['p_out'][index] for (scene,index) in zip(batch, indexs)]
    out = [np.concatenate((p[:,0],p[:,1])) for p in p_out]
    
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

train_loader = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0, drop_last=True)

In [ ]:
_, (example_datas, labels) = next(enumerate(train_loader))
example_datas.size()
#labels.size()

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = torch.nn.LSTM(4, 5, 4, dropout=0.2)        
        self.linear1 = torch.nn.Linear(5700, 1120)
        self.linear2 = torch.nn.Linear(1120, 60)

        self.drop = torch.nn.Dropout(0.2)
    def forward(self, inputs):
        x = inputs
        x = self.lstm(x)[0]
        x = x.reshape(1, -1)
        x = F.relu(self.linear1(x))
        x = self.drop(x)
        x = F.relu(self.linear2(x))
        return x.reshape(-1, 60)

In [ ]:
net = Net()
criterion = nn.MSELoss()
optimizer = optim.Adagrad(net.parameters(), lr=0.001)
a= 0
b = 0
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        a = labels
        b = outputs
        loss.backward()
        optimizer.step()

        # print statistics
        print(loss.item())
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [ ]:
LSTM = LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import itertools
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

batch_sz = 100

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    #inp = [np.dstack([scene['p_in'][:,:,0].reshape(60*19),scene['p_in'][:,:,1].reshape(60*19),scene['v_in'][:,:,0].reshape(60*19),scene['v_in'][:,:,1].reshape(60*19)]).reshape(60*19,4) for scene in batch]
    
    
    x_in = [np.concatenate([scene['p_in'][:,:,0].reshape(-1),scene['v_in'][:,:,0].reshape(-1), scene['lane'][:,0], scene['lane_norm'][:,0]]) for scene in batch]
    y_in = [np.concatenate([scene['p_in'][:,:,1].reshape(-1),scene['v_in'][:,:,1].reshape(-1), scene['lane'][:,1], scene['lane_norm'][:,1]]) for scene in batch]
    max_length = np.max([len(x) for x in x_in])
    diff_length = [max_length - len(x) for x in x_in]
    
    xs = [np.concatenate([x, np.zeros(diff)]).reshape(-1) for (x,diff) in zip(x_in, diff_length)]
    ys = [np.concatenate([y, np.zeros(diff)]).reshape(-1) for (y,diff) in zip(y_in, diff_length)]
    inp = [np.dstack([x,y]).reshape(-1,2) for (x, y) in zip(xs,ys)]
            
    indexs = [np.where(scene['agent_id'] == scene['track_id'][:,0].reshape(60))[0][0] for scene in batch]
    p_out = [scene['p_out'][index] for (scene,index) in zip(batch, indexs)]
    v_out = [scene['v_out'][index] for (scene,index) in zip(batch, indexs)]
    out = [np.concatenate((p[:,0],p[:,1],v[:,0],v[:,1])) for(p,v) in zip(p_out,v_out)]
    
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

train_loader = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0, drop_last=True)

In [ ]:
import itertools
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

batch_sz = 100

c

In [ ]:
_, (example_datas, labels) = next(enumerate(train_loader))
example_datas.size()
labels.size()

In [ ]:
_, (x_in, y_in, max_length, diff_length) = next(enumerate(train_loader))
[np.concatenate([x, np.zeros(diff)]) for (x,diff) in zip(x_in, diff_length)]
type(x_in[0])
#[np.concatenate(x, np.zeros(3)) for (x) in x_in]
np.concatenate([x_in[0], np.zeros(3)])

In [ ]:
agents = np.where(train_data[0]['car_mask'])[0]
p_in = train_data[0]['p_in'][agents]
v_in = train_data[0]['v_in'][agents]
v_in.shape
np.dstack([v_in[:,:,0].reshape(-1), v_in[:,:,0].reshape(-1)])

agents = [np.where(train_data[scene]['car_mask'])[0] for scene in [0,1,2]]
p_in = [train_data[scene]['p_in'][agent].reshape(-1,2) for (scene, agent) in zip([0,1,2], agents)]
p_in[0][:,0]

a = train_data[1]['lane'].shape[0]

b = np.zeros(270)[:a] = train_data[1]['lane'][:,0]
b

In [ ]:
max_lane = 0
max_lane_norm = 0
for scene in train_data: 
    if scene['lane'].shape[0] > max_lane: 
        max_line = scene['lane'].shape[0]
    if scene['lane_norm'].shape[0] > max_lane_norm:
        max_lane_norm = scene['lane_norm'].shape[0]


In [ ]:
class Encoder(nn.Module):

    def __init__(self, input_size, hidden_dim, num_layers=1):
        super(Encoder, self).__init__()

        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_dim, num_layers=self.num_layers)
        self.hidden = None

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))

    def forward(self, inputs):
        # Push through RNN layer (the ouput is irrelevant)
        _, self.hidden = self.lstm(inputs, self.hidden)
        return self.hidden

In [ ]:
class Decoder(nn.Module):

    def __init__(self, hidden_dim, num_layers=1):
        super(Decoder, self).__init__()
        # input_size=1 since the output are single values
        self.lstm = nn.LSTM(1, hidden_dim, num_layers=num_layers)
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, outputs, hidden, criterion):
        batch_size, num_steps = outputs.shape
        # Create initial start value/token
        input = torch.tensor([[0.0]] * batch_size, dtype=torch.float)
        # Convert (batch_size, output_size) to (seq_len, batch_size, output_size)
        input = input.unsqueeze(0)

        loss = 0
        for i in range(num_steps):
            # Push current input through LSTM: (seq_len=1, batch_size, input_size=1)
            #print(hidden[0].size())
            output, hidden = self.lstm(input, hidden)
            #print('here')
            # Push the output of last step through linear layer; returns (batch_size, 1)
            output = self.out(output[-1])
            # Generate input for next step by adding seq_len dimension (see above)
            input = output.unsqueeze(0)
            # Compute loss between predicted value and true value
            loss += criterion(output, outputs[:, i])
        return loss

In [ ]:
encoder = Encoder(2, 60)
decoder = Decoder(60)
criterion = nn.MSELoss()
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
a = 0
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        if (len(data[0]) < 4): 
            break
        inputs,labels = data
        inputs = inputs.transpose(1,0)
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        #print(inputs.shape[1])
        encoder.hidden = encoder.init_hidden(inputs.shape[1])
        #print(encoder.hidden[0].size())
        # Do forward pass through encoder
        hidden = encoder(inputs)
        #print(hidden[0].size())
        #a = hidden
        # Do forward pass through decoder (decoder gets hidden state from encoder)
        a = hidden
        
        loss = decoder(labels, hidden, criterion)
        # Backpropagation
        loss.backward()
        # Update parameters
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        running_loss += loss.item()
        print()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

# End of Lehan Li

In [ ]:
a[0].reshape(-1,120)

In [ ]:
if __name__ == '__main__':

    # 5 is the number of features of your data points

    # Create optimizers for encoder and decoder
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Some toy data: 2 sequences of length 10 with 5 features for each data point
    inputs = [
        [
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
        ],
        [
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
        ]
    ]

    inputs = torch.tensor(np.array(inputs), dtype=torch.float)
    # Convert (batch_size, seq_len, input_size) to (seq_len, batch_size, input_size)
    inputs = inputs.transpose(1,0)
    print(inputs.size())

    # 2 sequences (to match the batch size) of length 6 (for the 6h into the future)
    outputs = [ [0.1, 0.2, 0.3, 0.1, 0.2, 0.3], [0.3, 0.2, 0.1, 0.3, 0.2, 0.1] ]
    outputs = torch.tensor(np.array(outputs), dtype=torch.float)

    print(outputs.size())
    #
    # Do one complete forward & backward pass
    #
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Reset hidden state of encoder for current batch
    encoder.hidden = encoder.init_hidden(inputs.shape[1])
    # Do forward pass through encoder
    hidden = encoder(inputs)
    # Do forward pass through decoder (decoder gets hidden state from encoder)
    loss = decoder(outputs, hidden, criterion)
    # Backpropagation
    loss.backward()
    # Update parameters
    encoder_optimizer.step()
    decoder_optimizer.step()
    print("Loss:", loss.item())

### Visualize the batch of sequences

In [ ]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

        
for i_batch, sample_batch in enumerate(val_loader):
    inp, out = sample_batch
    """TODO:
      Deep learning model
      training routine
    """
    show_sample_batch(sample_batch, agent_id)
    break